In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

precision = 'FP64'
bag = 10
stack = 2

prefix = f'{precision}_B{bag}S{stack}'
dataset = 'oneToTwo_dataset_V100'
save_path = f'/mnt/disk1/gjh/AutoML_models/agtModel_{prefix}_{dataset}'
train = TabularDataset(f'{precision}Train.csv')
test = TabularDataset(f'{precision}Test.csv')


In [12]:
# import time
# agtPredictor = TabularPredictor.load(save_path)
# agtPredictor.persist_models(
#     models=['LightGBM_BAG_L1'], max_memory=0.5)
# tries = 200
# tmp = []
# s = time.perf_counter()
# for i in range(0, tries):
#     # s = time.perf_counter()
#     d = test.iloc[[i]]
#     t = agtPredictor.predict(d, model='LightGBM_BAG_L1')
#     # e = time.perf_counter()
#     # tmp.append((e-s)*1000)
# e = time.perf_counter()
# print(f"Run time per row: {(e-s)*1000/tries}ms")


In [2]:
agtPredictor = TabularPredictor(label='label', path=save_path, problem_type='multiclass',
                                learner_kwargs={'ignored_columns': ['Matrix']}).fit(
    train, num_bag_folds=bag, num_stack_levels=stack, ag_args_fit={'num_gpus': 0})


Beginning AutoGluon training ...
AutoGluon will save models to "/mnt/disk1/gjh/AutoML_models/agtModel_FP64_B10S5_oneToTwo_dataset_V100/"
AutoGluon Version:  0.4.0
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    4772
Train Data Columns: 12
Label Column: label
Preprocessing data ...
Train Data Class Count: 5
Using Feature Generators to preprocess the data ...
Dropping user-specified ignored columns: ['Matrix']
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    97758.47 MB
	Train Data (Original)  Memory Usage: 0.42 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in origi

In [14]:
# t = agtPredictor.persist_models(
#     models=agtPredictor.get_model_names(), max_memory=0.5)


In [3]:
agtPredictor = TabularPredictor.load(save_path)
a = agtPredictor.leaderboard(test, silent=True)
a10 = a.head(10)
a10 = a10[['model', 'score_test', 'score_val']]
a10plus = pd.DataFrame()
a10plus['model'] = a10['model']
best = a['model'][0]

In [4]:
score_train = []
for model in a10plus['model']:
    st = agtPredictor.evaluate(train, model=model, silent=True)
    score_train.append(st[agtPredictor.eval_metric.name])
    print(model)


RandomForestEntr_BAG_L2
NeuralNetTorch_BAG_L1
RandomForestGini_BAG_L2
NeuralNetFastAI_BAG_L2
RandomForestGini_BAG_L3
ExtraTreesGini_BAG_L3
ExtraTreesEntr_BAG_L1
LightGBMXT_BAG_L2
NeuralNetTorch_BAG_L2
RandomForestEntr_BAG_L3


In [5]:
a10plus['score_train'] = pd.Series(score_train)
af = a10.merge(a10plus)
af.to_csv(f'submit/{prefix}_{dataset}_score.csv', index=False)

In [25]:
# c1 = agtPredictor.predict(train, model=best)
# c1 = pd.DataFrame(c1)
# c1['matrix'] = train['matrix']
# c1['group'] = train['group']
# c1.to_csv(f'submit2/{prefix}_pred_on_train.csv', index=False)

# agtPredictor = TabularPredictor.load(save_path)
c2 = agtPredictor.predict(test, model=best)
c2 = pd.DataFrame(c2)
c2['Matrix'] = test['Matrix']
# c2['group'] = test['group']
c2.to_csv(f'submit/{prefix}_pred_on_test_P100.csv', index=False)


In [19]:
# # agtPredictor = TabularPredictor.load(save_path)
# a = agtPredictor.leaderboard(test, silent=True)
# b = a.head(10)[['model', 'pred_time_test', 'pred_time_test_marginal']]
# b.to_csv(f'submit2/{prefix}_pred_time_on_test.csv', index=False)
